In [1]:
import matplotlib.pyplot as plt
import glob
from skimage.color import rgb2lab
from skimage.transform import resize
from collections import namedtuple
import numpy as np
np.random.seed(101)
%matplotlib inline



In [2]:
import pickle
import os

training_file = 'train.p'
validation_file= 'valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

print()
print("Image Shape: {}".format(X_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_valid)))
print("Test Set:       {} samples".format(len(X_test)))


Image Shape: (32, 32, 3)

Training Set:   34799 samples
Validation Set: 4410 samples
Test Set:       12630 samples


In [3]:
### Preprocess the data here.
import cv2
import numpy as np 

# All the images were also Transform to grayscale, since the accuracy of the model was higher this way (and it shorten the model runtime)
# from shape (32,32,3) to (32,32,1)
def gray_scale(X_input):
    X_output = np.zeros((len(X_input),32,32,1))
    for i in range(len(X_input)):
        img = X_input[i].squeeze()
        img=img.astype(np.uint8)
        X_output[i,:,:,0] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return X_output


In [4]:
# Transform to gray
X_train_gray = gray_scale(X_train)
X_valid_gray = gray_scale(X_valid)
X_test_gray = gray_scale(X_test)

print(X_train_gray.shape)
print(X_valid_gray.shape)
print(X_test_gray.shape)


(34799, 32, 32, 1)
(4410, 32, 32, 1)
(12630, 32, 32, 1)


In [5]:
# Normalize images to [0,1]
X_train_norm= X_train_gray /255
X_valid_norm= X_valid_gray /255
X_test_norm= X_test_gray /255
# Shuffle 
from sklearn.utils import shuffle
X_train_shuffle, y_train_shuffle = shuffle(X_train_norm, y_train)
X_test_ready, y_test_ready = shuffle (X_test_norm, y_test)


# Split training and validation
n_train=len(X_train_shuffle)
limRange = int(n_train*0.8)

X_train_ready = (X_train_shuffle[0:limRange+1])
X_valid_ready = (X_train_shuffle[limRange+1:n_train+1])
y_train_ready = y_train_shuffle[0:limRange+1]
y_valid_ready = y_train_shuffle[limRange+1:n_train+1]


In [6]:
from keras.utils import np_utils

Using TensorFlow backend.


In [7]:
y_train_ready= np_utils.to_categorical(y_train_ready, 43)
y_valid_ready=np_utils.to_categorical(y_valid_ready, 43)

In [35]:
y_test_ready=np_utils.to_categorical(y_test_ready, 43)

In [8]:

X_train_ready.shape

(27840, 32, 32, 1)

In [9]:
X_train_ready.shape[1:]

(32, 32, 1)

In [10]:
X_train_ready = np.transpose(X_train_ready, (0,3, 1, 2))

In [29]:
X_valid_ready = np.transpose(X_valid_ready, (0,3, 1, 2))

In [33]:
X_test_ready = np.transpose(X_test_ready, (0,3, 1, 2))

In [11]:
X_train_ready.shape

(27840, 1, 32, 32)

In [16]:
X_train_ready.shape[1:]

(1, 32, 32)

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras import backend as K
 
def createModel():
    model = Sequential()
    model.add(Conv2D(6, (5, 5), padding='VALID', input_shape=X_train_ready.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation("relu"))
    model.add(Conv2D(16, (5, 5), padding='VALID'))
   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation("relu"))
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(84, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(43, activation='softmax'))
    return model

In [ ]:
from keras import backend as K
K.set_image_dim_ordering('th')

In [25]:
model1 = createModel()
batch_size = 256
epochs = 10
model1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
 


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [30]:
model1.fit(X_train_ready, y_train_ready,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_valid_ready, y_valid_ready))

Train on 27840 samples, validate on 6959 samples
Epoch 1/10
27840/27840 [==============================] - 48s - loss: 3.2350 - acc: 0.1492 - val_loss: 2.5999 - val_acc: 0.3580
Epoch 2/10
27840/27840 [==============================] - 50s - loss: 2.2406 - acc: 0.3761 - val_loss: 1.6099 - val_acc: 0.5606
Epoch 3/10
27840/27840 [==============================] - 51s - loss: 1.5424 - acc: 0.5366 - val_loss: 1.0391 - val_acc: 0.7370
Epoch 4/10
27840/27840 [==============================] - 49s - loss: 1.1009 - acc: 0.6710 - val_loss: 0.6934 - val_acc: 0.7993
Epoch 5/10
27840/27840 [==============================] - 51s - loss: 0.8081 - acc: 0.7585 - val_loss: 0.5007 - val_acc: 0.8825
Epoch 6/10
27840/27840 [==============================] - 47s - loss: 0.6229 - acc: 0.8119 - val_loss: 0.4168 - val_acc: 0.8869
Epoch 7/10
27840/27840 [==============================] - 45s - loss: 0.4996 - acc: 0.8520 - val_loss: 0.2931 - val_acc: 0.9294
Epoch 8/10
27840/27840 [==============================]

In [36]:
score = model1.evaluate(X_test_ready, y_test_ready, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.48233350260995816
Test accuracy: 0.8846397466255574


Augment the images
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('classifier.h5')